In [122]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
# Models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
# score
from sklearn.metrics import f1_score

train_ds=pd.read_csv(r'C:\\Users\\79375\\Desktop\\DS\\sber\\train.csv')
test_ds=pd.read_csv(r'C:\\Users\\79375\\Desktop\\DS\\sber\\sample_submission.csv')

test_ds['user_id'] = test_ds['id'].apply(lambda x: int(x.split(';')[0]))
test_ds['cart'] = test_ds['id'].apply(lambda x: int(x.split(';')[1]))
test_ds.drop('id', axis=1, inplace=True)
train_ds.drop('order_completed_at', axis=1, inplace=True)
train_ds['target'] = train_ds.duplicated(keep=0).apply(lambda x: 1 if x == 1 else 0)

In [123]:
id_tuple=pd.Series(train_ds['user_id'].drop_duplicates())
id_tuple=id_tuple.sort_values()

In [124]:
def procent(id_tuple_test):
    progress=id_tuple_test.count()
    i=0
    procent=[]
    while i <=100 :
        procent.append(int(round((progress/100)*i,0)))
        i+=1
    return procent

In [125]:
castomers_buy_map=[]
lap=1
process=procent(id_tuple)
for index in id_tuple:
    castomer_top_cart=train_ds[train_ds['user_id']==index]
    castomers_buy_map.append(castomer_top_cart)
    if index in process:
        clear_output()
        print(lap,' %')
        lap+=1

100  %


In [138]:
def cart_index_test_func(test_ds1):
    lap=1
    index=0
    process=procent(test_ds1['user_id'])
    cart_index_test=[]
    for i in range(test_ds['user_id'].count()):
        part_cart_index=[]
        part_cart_index.append(test_ds1.loc[i]['cart'])
        part_cart_index.append(test_ds1.loc[i]['user_id'])
        cart_index_test.append(part_cart_index)
        if index in process:
            clear_output()
            print(lap,' %')
            lap+=1
        index+=1
    return cart_index_test

In [139]:
cart_index_map=cart_index_test_func(test_ds)

100  %


In [140]:
id_tuple=pd.Series(train_ds['user_id'].drop_duplicates())
id_tuple=id_tuple.sort_values()

In [141]:
def castomeers_cart_map(id_tuple1):
    lap=1
    index=0
    process=procent(id_tuple1)
    customer_cart_index_target=[]
    for index_id in id_tuple1:
        customers_cart=train_ds[train_ds['user_id']==index_id]
        customer_cart_index_target.append(customers_cart)
        if index in process:
            clear_output()
            print(lap,' %')
            lap+=1
        index+=1
    return customer_cart_index_target

In [142]:
castomer_map=castomeers_cart_map(id_tuple)

100  %


In [174]:
predict_map=[]

process=procent(test_ds['user_id'])
lap=1
index_lap=0


top_carts=train_ds[['target','cart']].groupby('cart').sum().sort_values(by='target',ascending=False)
top_carts=top_carts[0:20]
top_carts=list(pd.Series(top_carts.index))
global_log=[]


for cart, index in cart_index_map:
    customers_cart=castomer_map[index]
    customers_cart=customers_cart['cart']
    customers_cart=list(pd.Series(customers_cart))
    
    castomer_target_cart=castomer_map[index]
    castomer_target_cart=castomer_target_cart[castomer_target_cart['target']==1]
    castomer_target_cart=castomer_target_cart['cart']
    castomer_target_cart=list(pd.Series(castomer_target_cart))

    if (cart in customers_cart)and(cart in castomer_target_cart)and(cart in top_carts):
        predict_map.append(1)
    elif (cart in customers_cart)and(cart in castomer_target_cart)and not(cart in top_carts):
        predict_map.append(1)
    elif (cart in customers_cart) and not(cart in castomer_target_cart) and not(cart in top_carts):
        predict_map.append(0)
    elif (cart in customers_cart) and not(cart in castomer_target_cart) and (cart in top_carts):        
        Castomet_cart_target_top=0
        for customers_cart_item in castomer_target_cart: 
            if customers_cart_item in top_carts:
                Castomet_cart_target_top+=1
        if len(castomer_target_cart)==0:
            if (cart in top_carts):
                predict_map.append(1)
        elif len(castomer_target_cart)!=0:
            predict_proc=Castomet_cart_target_top/len(castomer_target_cart)
            if predict_proc>=0.2:
                predict_map.append(1)
            elif predict_proc<0.2:
                predict_map.append(0)
        
    if index_lap in process:
        clear_output()
        print(lap,' %')
        lap+=1
    index_lap+=1

100  %


In [175]:
def scv_submiss(predictions, files_name='Standart_name'):
    id_cart=test_ds.user_id.apply(str) + ';' + test_ds.cart.apply(str)
    submission = pd.DataFrame({'id': id_cart, 'target': predictions})
    submission.to_csv(files_name+'.csv', index=False)

In [176]:
scv_submiss(predict_map, files_name='Standart_name 5.8')

In [177]:
f1_score(test_ds['target'],predict_map)

0.5382470983202189